## tf.data: 텐서플로우 입력 파이프라인 만들기
##### url: https://www.tensorflow.org/guide/data

- 목차
    - 기본 구조
        - 데이터셋 구조
    - 입력 데이터 불러오기
        - 넘파이 행렬 사용하기
        - 파이썬 제너레이터 사용하기
        - TFRecord 데이터 사용하기
        - 텍스트 데이터 사용하기
        - CSV 데이터 사용하기
        - 데이터 집합 사용하기
    - 데이터 원소 배치 만들기
        - 간단한 배치 만들기
        - 패딩과 함께 텐서 배치 만들기
    - 학습 흐름
        - 다중 에폭 수행하기
        - 입력 데이터를 임의로 섞기
    - 데이터 전처리
        - 이미지 데이터 복호화 및 리사이즈하기
        - 임의의 파이썬 함수 적용하기
        - tf.Example 프로토콜 버퍼 메시지 파싱하기
        - 시계열 윈도우 적용하기
        - 리샘플링
    - 고수준 API 사용하기
        - tf.keras
        - tf.estimator

`tf_data` API는 간단하고 재사용이 가능한 조각을 통해 복잡한 입력 파이프라인을 만들 수 있게 해줍니다.<br>
예를 들어 이미지 모델을 위한 파이프라인은 파일 시스템에 분산되어 있는 파일들을 모아주고, 각 이미지에 임의성을 부여하며 학습 배치로 사용될 선택된 이미지를 임의로 병합할 수 있습니다.<br>
텍스트 모델을 위한 파이프라인은 원본 텍스트 데이터에서 심볼을 추출하는 것을 포함해 룩업 테이블과 함께 임베딩 식별자로 변환하거나 서로 다른 길이를 가진 시퀀스로 배치를 만들 수 있습니다.<br>
`tf.data` API는 거대한 양의 데이터를 다룰 수 있게 만들어주며, 서로 다른 데이터 포맷을 읽거나 복잡한 변환을 수행할 수 있습니다.<br><br>
`tf.data` API는 하나 또는 그 이상의 컴포넌트로 구성된 각 원소의 시퀀스를 표현하는 `tf.data.Dataset` 추상화를 도입합니다. <br>
예를 들어, 이미지 파이프라인에서 이미지와 라벨로 표현된 텐서 컴포넌트의 쌍으로 원소들은 하나의 학습 샘플이 될 수 있습니다.

- 데이터 소스는 메모리에 저장되어 있거나 하나 또는 그 이상의 파일로 저장된 데이터로부터 `Dataset` 객체를 만듭니다.
- 데이터 변환은 하나 또는 그 이상의 `tf.data.Dataset` 객체로부터 데이터셋을 만듭니다.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
import tensorflow as tf

In [3]:
import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

np.set_printoptions(precision=4)

### 기본 구조
입력 파이프라인을 만드려면, 반드시 데이터 소스와 함께 시작해야 합니다. <br>
예를 들어, 메모리 내의 데이터로부터 `Dataset`를 만들기 위해서는 `tf.data.Dataset.from_tensors()` 함수 또는 `tf.data.Dataset.from_tensor_slices()`함수를 사용할 수 있습니다.<br>
대안으로, 입력 데이터가 권장되는 TFRecord 포맷의 파일로 저장되어 있다면 `tf.data.TFRecordDataset()` 함수를 사용할 수 있습니다.<br><br>한번 `Dataset` 객체를 만들게 되면, `tf.data.Dataset` 객체의 연쇄 메소드를 통해 새로운 `Dataset`으로 변환할 수 있습니다.<br>
예를 들어, 단일 원소에 적용되는 `Dataset.map()` 함수와 다중 원소에 적용되는 `Dataset.batch()` 변환을 수행할 수 있습니다.<br><br>
`Dataset` 객체는 '파이썬 이터러블(Iterable)' 객체이기 때문에 각 원소를 for loop을 사용해 모델에 입력할 수 있습니다.

In [4]:
dataset = tf.data.Dataset.from_tensor_slices([8, 3, 0, 8, 2, 1])
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [5]:
for elem in dataset:
    print(elem.numpy())

8
3
0
8
2
1


아니면 직접적으로 `iter` 함수를 사용해 파이썬 이터레이터를 사용하고 `next` 함수를 사용해 원소를 소진 합니다.

In [6]:
it = iter(dataset)

print(next(it).numpy())

8


대안으로 데이터셋 원소는 `reduce` 변환을 사용해 소진될 수 있으며, 해당 변환은 단일 결과를 생성하기위해 모든 원소를 압축합니다.<br>
다음 예제를 통해 데이터셋의 정수 합을 계산하는 `reduce` 변환이 어떻게 사용되는지 확인해보겠습니다.

In [7]:
print(dataset.reduce(0, lambda state, value: state + value).numpy())

22


#### 데이터셋 구조
데이터셋은 내재적으로 같은 구조를 가지고 있는 원소와 `Tensor`, `SparseTensor`, `ReggedTensor`, `TensorArray`, `Dataset`처럼 `tf.TypeSpec`에 의해 표현되는 타입 속성 구조를 가진 각각의 컴포넌트로 이루어져 있습니다.<br><br>
`Dataset.element_spec` 프로퍼티는 각 원소 컴포넌트의 타입을 검사할 수 있게 해줍니다. <br>
해당 프로퍼티는 원소의 구조를 매칭하기 위한 `tf.TypeSpec` 객체의 내부 내재된 구조를 반환하며 원소의 구조는 단일 컴포넌트, 튜플 컴포넌트 또는 내재된 튜풀 컴포넌트일 수도 있습니다.

In [8]:
# 단일 컴포넌트
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random.uniform([4, 10]))

dataset1.element_spec

TensorSpec(shape=(10,), dtype=tf.float32, name=None)

In [9]:
# 튜플 컴포넌트
dataset2 = tf.data.Dataset.from_tensor_slices(
    (tf.random.uniform([4]),
     tf.random.uniform([4, 100], maxval=100, dtype=tf.int32))
)

dataset2.element_spec

(TensorSpec(shape=(), dtype=tf.float32, name=None),
 TensorSpec(shape=(100,), dtype=tf.int32, name=None))

In [10]:
# 내재된 튜플 컴포넌트
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))

dataset3.element_spec

(TensorSpec(shape=(10,), dtype=tf.float32, name=None),
 (TensorSpec(shape=(), dtype=tf.float32, name=None),
  TensorSpec(shape=(100,), dtype=tf.int32, name=None)))

In [11]:
# 희소 텐서를 포함하는 데이터셋
dataset4 = tf.data.Dataset.from_tensors(
   tf.SparseTensor(indices=[[0, 0], [1, 2]],
                   values=[1, 2],
                   dense_shape=[3, 4]
                  )
)
dataset4.element_spec

SparseTensorSpec(TensorShape([3, 4]), tf.int32)

In [12]:
# value_type를 사용해서 element spec에 의해 표현되는 값의 타입 살펴보기
dataset4.element_spec.value_type

tensorflow.python.framework.sparse_tensor.SparseTensor

`Dataset` 변환은 모든 구조의 데이터셋을 지원합니다. <br>
각 원소에 적용 되는 함수인 `Dataset.map()` 또는 `Dataset.filter()` 변환을 사용할 떄, 원소 구조는 함수의 인자로 결정됩니다.

In [13]:
dataset1 = tf.data.Dataset.from_tensor_slices(
    tf.random.uniform([4, 10], minval=1, maxval=10, dtype=tf.int32)
)
dataset1

<TensorSliceDataset shapes: (10,), types: tf.int32>

In [14]:
for z in dataset1:
    print(z.numpy())

[9 4 6 2 7 4 7 3 1 1]
[9 5 4 3 5 4 6 7 7 9]
[1 4 6 5 2 1 1 8 6 8]
[5 7 8 8 7 5 9 7 1 8]


In [15]:
dataset2 = tf.data.Dataset.from_tensor_slices(
    (tf.random.uniform([4]),
     tf.random.uniform([4, 100], maxval=100, dtype=tf.int32)
    
    )
)

dataset2

<TensorSliceDataset shapes: ((), (100,)), types: (tf.float32, tf.int32)>

In [16]:
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))

dataset3

<ZipDataset shapes: ((10,), ((), (100,))), types: (tf.int32, (tf.float32, tf.int32))>

In [17]:
for a, (b, c) in dataset3:
    print(f'shapes: {a.shape}, {b.shape}, {c.shape}')

shapes: (10,), (), (100,)
shapes: (10,), (), (100,)
shapes: (10,), (), (100,)
shapes: (10,), (), (100,)


### 입력 데이터 읽어오기
#### 넘파이 행렬 읽어오기
더 많은 예제는 [Loading NumPy arrays](https://www.tensorflow.org/tutorials/load_data/numpy)를 확인하세요.<br><br>
만약 모든 입력 데이터가 메모리에 올라갈 수 있다면, `Dataset`을 만드는 가장 쉬운 방법은 `tf.Tensor` 객체로 변환하거나, `Dataset.from_tensor_slices()` 함수를 사용하는 것입니다.

In [18]:
train, test = tf.keras.datasets.fashion_mnist.load_data()

In [19]:
images, labels = train
images = images/255.0

dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset

<TensorSliceDataset shapes: ((28, 28), ()), types: (tf.float64, tf.uint8)>

> Note: 위의 코드 스니펫은 변수(feature)와 라벨(label) 행렬을 `tf.constant()` 연산으로써 텐서플로우 그래프에 포함합니다. 이러한 작업은 작은 데이터셋에는 유용할 수 있으나, 여러번에 걸쳐 행렬의 내용을 복사하므로 메모리를 낭비할 수 있습니다. 그리고 2GB로 제한된 `tf.GraphDef` 프로토콜 버퍼의 크기에 도달할 수 있습니다.

#### 파이썬 제너레이터로 읽어오기
다른 `tf.data.Dataset`으로써 쉽게 수집될 수 있는 일반적인 데이터 소스는 파이썬 제너레이터가 있습니다.

> 주의: 이것은 이식성과 확장성을 제한할 수 있는 가장 편리한 접근 방법이지만, 이러한 작업은 반드시 제너레이터를 생성한 같은 파이썬 프로세스에서 실행되어야하며, 여전히 파이썬 GIL이 적용됩니다.

In [20]:
def count(stop):
    i = 0
    while i<stop :
        yield i
        i += 1

In [21]:
for n in count(5):
    print(n)

0
1
2
3
4


`Dataset.from_generator` 생성자는 파이썬 제너레이터를 완전히 함수적인 `tf.data.Dataset`으로 변환합니다.<br><br>
그 생성자는 입력으로써 iterator가 아닌 callable을 사용합니다. 이를 통해 제너레이터가 끝에 다다랐을 때 재시작될 수 있도록 합니다.<br>
선택적으로 callable의 인자로서 넘겨줄 수 있도록 `args` 인수를 사용할 수 있습니다.<br><br>
`output_types` 인수는 내부적으로 `tf.data`가 `tf.Graph`를 만들기 때문에 요구됩니다. 그리고 그래프 엣지는 `tf.dtype`을 요구합니다.

In [22]:
ds_counter = tf.data.Dataset.from_generator(count,
                                            args=[25],
                                            output_types=tf.int32,
                                            output_shapes = (), )

In [23]:
for count_batch in ds_counter.repeat().batch(10).take(10):
    print(count_batch.numpy())

[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24  0  1  2  3  4]
[ 5  6  7  8  9 10 11 12 13 14]
[15 16 17 18 19 20 21 22 23 24]
[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24  0  1  2  3  4]
[ 5  6  7  8  9 10 11 12 13 14]
[15 16 17 18 19 20 21 22 23 24]


`outpyt_shapes` 인수는 요구되지 않습니다. 하지만 많은 텐서플로우 연산들이 알려지지 않은(unknown) 랭크를 지원하지 않기 때문에 입력하는 것을 강력히 권해드립니다.<br>
만약 특정한 축의 길이가 알려지지 않거나 또는 변할 수 있다면, `output_shape`에 `None` 을 세팅합니다.<br><br>
또한, `output_shapes`와 `output_types`는 다른 데이터 메소드와서도 동일한 중첩 규칙을 따른다는 것도 중요합니다.<br><br>
여기에 두 측면을 살펴볼 제너레이터 예제가 있습니다. 이 제너레이터는 행렬의 튜플을 반환하며, 두번째 행렬은 길이가 알려지지 않은 벡터입니다.

In [24]:
def gen_series():
    i = 0
    while True:
        size = np.random.randint(0, 10)
        yield i, np.random.normal(size=(size,))
        i += 1

In [25]:
for i, series in gen_series():
    print(i, ':', str(series))
    if i > 5:
        break

0 : [3.0212 0.6412]
1 : [-0.9376 -0.759   1.5679]
2 : []
3 : []
4 : [-1.38    0.2749 -1.3062 -0.5613 -0.7814  0.6736  1.6538]
5 : [-0.9012  1.4923  0.8746 -0.5763  0.8752]
6 : [0.8371 0.0814 0.8715 1.2851 0.3053]


첫 번째 출력 값은 `int32`이고, 두 번째 값은 `float32`<br><br>
첫 번째 아이템은 `()` 차원의 스칼라이며, 두 번째는 알려지지 않은 길이인 `(None, )`차원의 벡터 입니다.

In [26]:
ds_series = tf.data.Dataset.from_generator(
    gen_series,
    output_types=(tf.int32, tf.float32),
    output_shapes=((), (None)))

ds_series

<DatasetV1Adapter shapes: ((), <unknown>), types: (tf.int32, tf.float32)>

이제 일반적인 `tf.data.Dataset`처럼 사용될 수 있습니다. 변화할 수 있는 차원을 가진 데이터셋을 배치로 만들 때, `Dataset.padded_batch`를 사용할 필요가 있습니다.

In [27]:
# padded_batch를 사용해서 배치 사이즈를 통해 일정한 길이로 맞춰줄 수 있습니다.
ds_series_batch = ds_series.shuffle(20).padded_batch(10, padded_shapes=([], [None]))

ids, sequence_batch = next(iter(ds_series_batch))
print(ids.numpy(), end='\n\n')
print(sequence_batch.numpy())

[14  5 19 22  0  8  6 10  2 16]

[[-3.0765 -0.5676  0.      0.      0.      0.      0.    ]
 [ 0.9806  0.7378  0.      0.      0.      0.      0.    ]
 [ 0.0386 -0.4955 -1.8543  0.      0.      0.      0.    ]
 [ 1.1859  0.      0.      0.      0.      0.      0.    ]
 [-0.1616 -0.1837 -0.538   1.2159  0.6029  1.2339  0.    ]
 [-0.7248 -0.9983  0.1881  0.      0.      0.      0.    ]
 [-0.7503 -0.0452  0.1803  0.      0.      0.      0.    ]
 [ 0.4174  0.5916 -0.6174  0.8604  0.6191 -0.4476 -0.9568]
 [ 0.6094 -0.1896 -0.134  -1.0852  0.1142 -0.6709 -2.6041]
 [-1.6669  0.714  -2.0217  0.509   1.0485 -0.4258  1.3149]]


더 현실적인 예제로, `tf.data.Dataset`로써 `preprocessing.image.ImageDataGenerator`로 래핑을 시도해보겠습니다.<br><br>
먼저 데이터를 다운받습니다.

In [28]:
flowers = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True
)

`image.ImageDataGenerator`를 생성합니다.

In [29]:
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, rotation_range=20)

In [30]:
images, labels = next(img_gen.flow_from_directory(flowers))

Found 3670 images belonging to 5 classes.


In [31]:
print(images.dtype, images.shape)
print(labels.dtype, labels.shape)

float32 (32, 256, 256, 3)
float32 (32, 5)


In [36]:
ds = tf.data.Dataset.from_generator(
    img_gen.flow_from_directory, 
    args=[flowers],
    output_types=(tf.float32, tf.float32), 
    output_shapes=([32,256,256,3], [32,5])
)

ds

<DatasetV1Adapter shapes: ((32, 256, 256, 3), (32, 5)), types: (tf.float32, tf.float32)>

### TFRecord 데이터 사용하기

End-to-end 예제를 위해 [Loading TFRecords](https://www.tensorflow.org/tutorials/load_data/tf_records)를 살펴보세요.<br><br>
`tf.data` API는 다양한 파일 포맷을 지원합니다. 그래서 데이터 셋이 메모리에 전부 적재되지 않더라도 크기가 큰 데이터셋을 처리할 수 있습니다.<br>
예를 들어, TFRecord 파일 포맷은 많은 텐서플로우 어플리케이션이 학습 데이터에 사용하는 간단한 레코드-지향 이진 포맷입니다.<br>
`tf.data.TFRecordDataset` 클래스는 입력 파이프라인의 일부로써 하나 또는 그 이상의 TFRecord 파일의 컨텐츠를 스트림할 수 있도록 해줍니다.<br><br>
French Street Name Signs(FSNS) 데이터셋의 테스트 파일을 사용한 예제를 살펴보겠습니다.

In [37]:
# 두 개의 파일의 예제의 전부를 불러와 데이터셋을 생성합니다.
fsns_test_file = tf.keras.utils.get_file("fsns.tfrec", 
                                         "https://storage.googleapis.com/download.tensorflow.org/data/fsns-20160927/testdata/fsns-00000-of-00001")

7905280/7904079 [==============================] - 0s 0us/step


`tf.data.TFRecordDataset`의 `filename` 인자는 문자열이거나 문자열의 리스트 또는, `tf.Tensor` 문자열일 수도 있습니다.<br>
그러므로 학습과 검증의 목적으로 두개로 이루어진 파일을 가지고 있다면, 입력 인자로써 파일 이름을 사용해 데이터셋을 생성하기 위한 팩토리 메소드를 생성할 수 있습니다.

In [38]:
dataset = tf.data.TFRecordDataset(filenames = [fsns_test_file])
dataset

<TFRecordDatasetV2 shapes: (), types: tf.string>

많은 텐서플로우 프로젝트는 TFRecord 파일에 있는 직렬화된 `tf.train.Example` 레코드를 사용합니다.<br>
해당 레코드들은 확인되기 전에, 복호화가 필요합니다.

In [43]:
raw_example = next(iter(dataset))
parsed = tf.train.Example.FromString(raw_example.numpy())

parsed.features.feature['image/text']

bytes_list {
  value: "Rue Perreyon"
}

### 텍스트 데이터 사용하기

End-to-End 예제를 위해 [Loading Text](https://www.tensorflow.org/tutorials/load_data/text)를 살펴보세요.<br><br>
많은 데이터셋이 하나 또는 그 이상의 텍스트 파일로 분산되어 있습니다.<br>
`tf.data.TextLineDataset`은 하나 또는 이상의 텍스트 파일로부터 행을 추출하는 가장 쉬운 방법을 제공합니다을<br>
하나 또는 더 많은 파일 이름이 주어지면, `TextLineDataset`은 해당 파일의 행 당 하나의 문자열-값 원소를 생성하게됩니다.

In [46]:
directory_url = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
file_names = ['cowper.txt', 'derby.txt', 'butler.txt']

file_paths = [
    tf.keras.utils.get_file(file_name, directory_url + file_name) for file_name in file_names
]

811008/807992 [==============================] - 0s 0us/step


In [48]:
dataset = tf.data.TextLineDataset(file_paths)

첫 번째 파일의 앞의 몇 행을 확인해보겠습니다.

In [49]:
for line in dataset.take(5):
    print(line.numpy())

b"\xef\xbb\xbfAchilles sing, O Goddess! Peleus' son;"
b'His wrath pernicious, who ten thousand woes'
b"Caused to Achaia's host, sent many a soul"
b'Illustrious into Ades premature,'
b'And Heroes gave (so stood the will of Jove)'


여러 파일들 사이에서 행이 번갈아 일어나도록 `Dataset,interleave` 함수를 사용합니다.<br>
이는 파일이 손쉽게 섞일 수 있도록 만들어줍니다.<br>

In [55]:
files_ds = tf.data.Dataset.from_tensor_slices(file_paths)
lines_ds = files_ds.interleave(tf.data.TextLineDataset, cycle_length=3)

for i, line in enumerate(lines_ds.take(9)):
    if i % 3 == 0:
        print()
    print(line.numpy())


b"\xef\xbb\xbfAchilles sing, O Goddess! Peleus' son;"
b"\xef\xbb\xbfOf Peleus' son, Achilles, sing, O Muse,"
b'\xef\xbb\xbfSing, O goddess, the anger of Achilles son of Peleus, that brought'

b'His wrath pernicious, who ten thousand woes'
b'The vengeance, deep and deadly; whence to Greece'
b'countless ills upon the Achaeans. Many a brave soul did it send'

b"Caused to Achaia's host, sent many a soul"
b'Unnumbered ills arose; which many a soul'
b'hurrying down to Hades, and many a hero did it yield a prey to dogs and'


기본 값으로 `TextLineDataset`은 각 파일의 모든 행을 생성하므로 파일이 주석을 포함하고 있거나 헤더 라인부터 시작된다면, 이는 바람직하지 않을 수 있습니다.<br>
그런 행들은 `Dataset.skip()` 또는 `Dataset.filter()` 변환을 사용하여 제거할 수 있습니다.<br>
첫 번째 행을 건너뛰고 생존자만을 필터링하겠습니다.

In [56]:
titanic_file = tf.keras.utils.get_file("train.csv", "https://storage.googleapis.com/tf-datasets/titanic/train.csv")
titanic_lines = tf.data.TextLineDataset(titanic_file)

32768/30874 [===============================] - 0s 1us/step


In [57]:
for line in titanic_lines.take(10):
    print(line.numpy())

b'survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone'
b'0,male,22.0,1,0,7.25,Third,unknown,Southampton,n'
b'1,female,38.0,1,0,71.2833,First,C,Cherbourg,n'
b'1,female,26.0,0,0,7.925,Third,unknown,Southampton,y'
b'1,female,35.0,1,0,53.1,First,C,Southampton,n'
b'0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y'
b'0,male,2.0,3,1,21.075,Third,unknown,Southampton,n'
b'1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n'
b'1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n'
b'1,female,4.0,1,1,16.7,Third,G,Southampton,n'


In [58]:
def survived(line):
    return tf.not_equal(tf.strings.substr(line, 0, 1), '0')

survivors = titanic_lines.skip(1).filter(survived)

In [59]:
for line in survivors.take(10):
    print(line.numpy())

b'1,female,38.0,1,0,71.2833,First,C,Cherbourg,n'
b'1,female,26.0,0,0,7.925,Third,unknown,Southampton,y'
b'1,female,35.0,1,0,53.1,First,C,Southampton,n'
b'1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n'
b'1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n'
b'1,female,4.0,1,1,16.7,Third,G,Southampton,n'
b'1,male,28.0,0,0,13.0,Second,unknown,Southampton,y'
b'1,female,28.0,0,0,7.225,Third,unknown,Cherbourg,y'
b'1,male,28.0,0,0,35.5,First,A,Southampton,y'
b'1,female,38.0,1,5,31.3875,Third,unknown,Southampton,n'
